## Retention Years

In [148]:
df_orders = pd.read_csv('/content/orders.csv')
df_orders['order_month'] = df_orders['order_date'].apply(lambda x: x[0:7])
df_orders['order_year'] = df_orders['order_date'].apply(lambda x: x[0:4]).astype(int)
#df_orders['month'] = df_orders['order_date'].apply(lambda x: x[5:7])


In [137]:
df_customers = pd.read_csv('/content/customers.csv')
cols = ['customer_id']
cols.extend(df_customers.columns[1:])
df_customers.columns = cols

In [149]:
df_merge = pd.merge(df_orders, df_customers, 'inner', on='customer_id')

In [150]:
df_first = df_merge.groupby('customer_id')['order_year'].min().reset_index()
df_first.columns = ['customer_id', 'first_order']

In [151]:
df_full = pd.merge(df_merge, df_first, 'inner', on='customer_id')

In [152]:
df_users = df_full.groupby(['customer_id', 'first_order', 'order_year', 'ship_mode', 'segment'])\
    .agg({'sales':'sum', 'id':'count'}).reset_index()

In [162]:
df_users

,customer_id,first_order,order_year,ship_mode,segment,sales,id
0,AA-10315,2014,2014,Standard,Consumer,756.048,2
1,AA-10315,2014,2015,First,Consumer,26.960,1
2,AA-10315,2014,2016,Standard,Consumer,4406.072,1
3,AA-10315,2014,2017,Standard,Consumer,374.480,1
4,AA-10375,2014,2014,Standard,Consumer,50.792,2
...,...,...,...,...,...,...,...
3680,ZC-21910,2014,2017,Standard,Consumer,183.146,3
3681,ZD-21925,2014,2014,Second,Consumer,244.760,1
3682,ZD-21925,2014,2016,Same Day,Consumer,839.944,1
3683,ZD-21925,2014,2016,Standard,Consumer,347.800,2


In [153]:
def get_crosstab(df, value_column):
    df_crosstab = pd.crosstab(df.customer_id, df.order_year, values=df[value_column], aggfunc=sum)\
        .reset_index()
    df_crosstab = pd.merge(df_crosstab, df_first, how='inner', on='customer_id')
    del df_crosstab['customer_id']
    return df_crosstab

def get_crosstab_cur(df, value_column):
    df_crosstab = pd.crosstab(df.customer_id, df.order_year, values=df[value_column], aggfunc=sum)\
        .reset_index()
    df_crosstab = pd.merge(df_crosstab, df_first, how='inner', on='customer_id')
    del df_crosstab['customer_id']
    return df_crosstab
def get_crosstab_last(df, value_column):
    df_crosstab = pd.crosstab(df.customer_id, df.order_year, values=df[value_column], aggfunc=sum)\
        .reset_index()
    df_crosstab = pd.merge(df_crosstab, df_first, how='inner', on='customer_id')
    del df_crosstab['customer_id']
    return df_crosstab

In [154]:
def get_retention(df):
    df_triangle = df.groupby(['first_order']).sum().reset_index()
    df_triangle['Base'] = np.diag(df_triangle.iloc[:,1:])
    df_retention = pd.DataFrame(df_triangle[['first_order', 'Base']])
    df_retention.columns = ['Cohort', 'Base']

    for col in df_triangle.columns[1:-1]:
        df_retention[col] = round(df_triangle[col] / df_triangle['Base'], 3)

    for i in range(2, len(df_retention.columns)):
        df_retention.iloc[i-1:, i] = None
        
    return df_retention

### Sales retention

In [155]:
get_retention(get_crosstab(df_users, 'sales'))

,Cohort,Base,2014,2015,2016,2017
0,2014,484247.4981,1.0,0.754,0.939,1.101
1,2015,105388.4962,NaN,1.000,0.944,1.250
2,2016,54803.6200,NaN,NaN,1.000,1.111
3,2017,7511.8050,NaN,NaN,NaN,1.000


In [182]:
1-7511.8050/54803.6200

0.8629323208941307

### Transaction retention

In [157]:
get_retention(get_crosstab(df_users, 'id'))

,Cohort,Base,2014,2015,2016,2017
0,2014,969.0,1.0,0.833,1.028,1.288
1,2015,231.0,NaN,1.000,0.931,1.268
2,2016,104.0,NaN,NaN,1.000,1.221
3,2017,19.0,NaN,NaN,NaN,1.000


In [183]:
1 - 19/104

0.8173076923076923

### Sales retention by segment

In [158]:
for segment in df_users.segment.unique():
    print(f'\nSegment: {segment}')
    print(get_retention(get_crosstab(df_users[df_users.segment==segment], 'sales')))


Segment: Consumer
   Cohort         Base  2014   2015   2016   2017
0    2014  266096.8126   1.0  0.789  0.818  0.906
1    2015   56590.8551   NaN  1.000  0.832  0.817
2    2016   32119.4550   NaN    NaN  1.000  1.229
3    2017    4985.0760   NaN    NaN    NaN  1.000

Segment: Home Office
   Cohort        Base  2014   2015   2016   2017
0    2014  89715.8118   1.0  0.616  0.874  1.296
1    2015  19942.1216   NaN  1.000  1.120  1.475
2    2016   4483.5420   NaN    NaN  1.000  2.905
3    2017    729.6480   NaN    NaN    NaN  1.000

Segment: Corporate
   Cohort         Base  2014   2015   2016   2017
0    2014  128434.8737   1.0  0.778  1.237  1.367
1    2015   28855.5195   NaN  1.000  1.043  1.944
2    2016   18200.6230   NaN    NaN  1.000  0.462
3    2017    1797.0810   NaN    NaN    NaN  1.000


### Sales retention by ship_mode

In [160]:
for ship_mode in df_users.ship_mode.unique():
    print(f'\nShip_mode: {ship_mode}')
    print(get_retention(get_crosstab(df_users[df_users.ship_mode==ship_mode], 'sales')))


Ship_mode: Standard
   Cohort         Base  2014   2015   2016   2017
0    2014  305621.3196   1.0  0.716  0.921  0.935
1    2015   65723.8550   NaN  1.000  0.791  1.142
2    2016   38855.4240   NaN    NaN  1.000  0.752
3    2017    5644.8960   NaN    NaN    NaN  1.000

Ship_mode: First
   Cohort        Base  2014   2015   2016   2017
0    2014  59769.2645   1.0  0.997  1.010  1.590
1    2015   9644.0606   NaN  1.000  1.632  2.143
2    2016   6153.2850   NaN    NaN  1.000  3.814
3    2017    955.7840   NaN    NaN    NaN  1.000

Ship_mode: Second
   Cohort         Base  2014   2015   2016   2017
0    2014  101386.7800   1.0  0.642  0.811  1.105
1    2015   23970.2056   NaN  1.000  1.180  1.216
2    2016    9537.8710   NaN    NaN  1.000  0.697
3    2017     911.1250   NaN    NaN    NaN  1.000

Ship_mode: Same Day
   Cohort       Base  2014   2015   2016   2017
0    2014  17470.134   1.0  1.234  1.759  2.308
1    2015   6050.375   NaN  1.000  0.581  1.136
2    2016    257.040   NaN    Na

### Transaction retention by segment

In [161]:
for segment in df_users.segment.unique():
    print(f'\nSegment: {segment}')
    print(get_retention(get_crosstab(df_users[df_users.segment==segment], 'id')))


Segment: Consumer
   Cohort   Base  2014   2015   2016   2017
0    2014  524.0   1.0  0.817  0.954  1.265
1    2015  107.0   NaN  1.000  0.869  1.215
2    2016   58.0   NaN    NaN  1.000  1.207
3    2017   13.0   NaN    NaN    NaN  1.000

Segment: Home Office
   Cohort   Base  2014   2015   2016   2017
0    2014  158.0   1.0  0.899  1.095  1.405
1    2015   49.0   NaN  1.000  1.122  1.429
2    2016   14.0   NaN    NaN  1.000  1.786
3    2017    1.0   NaN    NaN    NaN  1.000

Segment: Corporate
   Cohort   Base  2014   2015   2016   2017
0    2014  287.0   1.0  0.826  1.125  1.265
1    2015   75.0   NaN  1.000  0.893  1.240
2    2016   32.0   NaN    NaN  1.000  1.000
3    2017    5.0   NaN    NaN    NaN  1.000


### Transaction retention by ship_mode

In [ ]:
for ship_mode in df_users.ship_mode.unique():
    print(f'\nShip_mode: {ship_mode}')
    print(get_retention(get_crosstab(df_users[df_users.ship_mode==ship_mode], 'id')))


Ship_mode: Standard
  Cohort   Base  2014   2015   2016   2017
0   2014  586.0   1.0  0.845  1.002  1.229
1   2015  141.0   NaN  1.000  0.943  1.284
2   2016   62.0   NaN    NaN  1.000  1.242
3   2017   12.0   NaN    NaN    NaN  1.000

Ship_mode: First
  Cohort   Base  2014   2015   2016   2017
0   2014  145.0   1.0  0.772  1.103  1.448
1   2015   31.0   NaN  1.000  1.194  1.516
2   2016   18.0   NaN    NaN  1.000  1.278
3   2017    4.0   NaN    NaN    NaN  1.000

Ship_mode: Second
  Cohort   Base  2014   2015   2016   2017
0   2014  190.0   1.0  0.826  0.995  1.311
1   2015   49.0   NaN  1.000  0.673  1.020
2   2016   22.0   NaN    NaN  1.000  1.000
3   2017    3.0   NaN    NaN    NaN  1.000

Ship_mode: Same Day
  Cohort  Base  2014   2015  2016   2017
0   2014  48.0   1.0  0.896  1.25  1.438
1   2015  10.0   NaN  1.000  1.20  1.500
2   2016   2.0   NaN    NaN  1.00  2.500


## Monthly Retention

In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
from operator import attrgetter

In [3]:
orders = pd.read_csv('/content/orders.csv', encoding = "ISO-8859-1")
customers = pd.read_csv('/content/customers.csv', encoding = "ISO-8859-1")

In [4]:
orders['order_date'] = pd.to_datetime(orders['order_date'])
orders['order_y_m'] = orders['order_date'].map(lambda date: 100*date.year + date.month)

In [24]:
orders['order_date_months'] = orders['order_date'].map(lambda date: 12*date.year + date.month)

In [32]:
orders_fp = orders.groupby('customer_id').order_y_m.min().reset_index()
orders_fp.columns = ['customer_id', 'first_order']
#orders_fp['first_order_months'] = orders_fp['first_order'].astype(str)[:3]*12+orders_fp['first_order'].astype(str)[3:]

In [34]:
orders_fp_m = orders.groupby('customer_id').order_date_months.min().reset_index()
orders_fp_m.columns = ['customer_id', 'first_order_m']

In [6]:
orders = pd.merge(orders, orders_fp, how='inner', on='customer_id')

In [36]:
orders = pd.merge(orders, orders_fp_m, how='inner', on='customer_id')

In [55]:
#df_users = orders.groupby(['customer_id', 'first_order', 'order_y_m'])\
#    .agg({'sales':'sum'}).reset_index()

"""
df_users = orders.groupby(['customer_id', 'first_order', 'order_y_m'])\
    .agg({'sales':'sum', 'id':'count'}).reset_index()

df_users['period_number'] = df_users['order_y_m']//100 *12 + df_users['order_y_m']%100 - df_users['first_order']//100 *12 - df_users['first_order']%100
"""

df_users = orders.groupby(['first_order', 'order_y_m'])\
    .agg({'sales':'sum', 'id':'count'}).reset_index()

df_users['period_number'] = df_users['order_y_m']//100 *12 + df_users['order_y_m']%100 - df_users['first_order']//100 *12 - df_users['first_order']%100


In [164]:
df_users_pivot_sales = df_users.pivot_table(index = 'first_order',
                                     columns = 'period_number',
                                     values = 'sales')

KeyError: ignored

In [ ]:
cohort_size_n = df_users_pivot_n.sum(axis=1)
retention_n = df_users_pivot_n.div(cohort_size_n, axis = 0)

In [175]:
sales_2016 = df_users_pivot_sales.iloc[12+12:12+12+11, :12]
base_sales_2016=sales_2016.iloc[:,0]
retention_sales_2016 = sales_2016.div(base_sales_2016, axis = 0)

In [176]:
sales_2017 = df_users_pivot_sales.iloc[12+12+11:12+12+12+12, :12]
base_sales_2017=sales_2017.iloc[:,0]
retention_sales_2017 = sales_2017.div(base_sales_2017, axis = 0)

In [177]:
retention_sales_2016

period_number,0,1,2,3,4,5,6,7,8,9,10,11
first_order,,,,,,,,,,,,
201601,1.0,0.218531,NaN,0.014558,NaN,0.008271,0.007332,0.032950,NaN,0.008652,0.025529,0.044854
201602,1.0,NaN,1.891543,NaN,NaN,NaN,NaN,NaN,35.444955,1.313166,NaN,5.770368
201603,1.0,NaN,3.691797,NaN,0.164859,NaN,0.103889,0.170551,0.103954,0.299546,0.704617,0.174636
201604,1.0,0.040636,NaN,0.103116,0.042646,0.321080,NaN,0.008928,NaN,0.168894,NaN,4.948938
201605,1.0,NaN,0.250093,0.129575,0.001744,0.008012,0.235403,0.055689,0.172719,0.022908,0.038521,0.058344
201606,1.0,0.502826,2.145873,0.239481,0.030234,0.822022,0.006198,1.166255,NaN,0.045057,NaN,NaN
201607,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.018134,NaN,NaN
201608,1.0,NaN,NaN,NaN,NaN,0.624534,NaN,0.046205,0.167169,0.455075,0.044171,NaN
201610,1.0,NaN,20.410758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.236145,0.017767


In [178]:
retention_sales_2017

period_number,0,1,2,3,4,5,6,7,8,9,10,11
first_order,,,,,,,,,,,,
201703,1.0,NaN,NaN,0.013133,1.152067,0.432962,NaN,1.391345,NaN,NaN,NaN,NaN
201704,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201706,1.0,NaN,NaN,NaN,NaN,NaN,1.372686,NaN,NaN,NaN,NaN,NaN
201707,1.0,NaN,NaN,NaN,NaN,0.174213,NaN,NaN,NaN,NaN,NaN,NaN
201709,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201710,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201711,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Ответ
#### 1. Суммарно продажи и число новых покупателей ниже в 2017 году ниже, чем в 2016: продажи упали на 86%, пророст покапателей -- на 81% (Retention Years)
#### 2. Также ретеншен в 2016 году лучше, чем в 2017: в 2017 году ретеншен не нулевой только для некоторых месяце (трех), а в 2016 повротные покупки новых пользователей наблюдались почти в каждом месяце.

